In [1]:
import tweepy
import csv
import pandas as pd
import time
import sys
import itertools
from datetime import datetime
import sys
import networkx as nx
import plotly as py
import plotly.graph_objects as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Twitter API credentials
#consumer_key = "BmvoUUcOUXhPxRR8uRC2TgKoW"
#consumer_secret = "bNV6inRgeUSSVerytnnnTPveW8iM9GM0dwryZyiUKmYy436D1I"
#access_key = "2969993776-b9Ui7fVJjW7gYId2C0kSGo5mN4ki93HSGEn6jx0"
#access_secret = "N5ER33zjeIqfl5918MWTHLWbZzuBGfGL0FeSfNGvSsrvZ"

In [3]:
#OAUTH_KEYS = {'consumer_key':consumer_key, 'consumer_secret':consumer_secret, 'access_token_key':access_key, 'access_token_secret':access_secret}
#auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
def get_names_for_download(dates):
    pwd = '/home/b00762654/twitter_None'
    for i in range(len(dates) - 1):
        csv_name = pwd + '/data_'+ str(dates[i]) + '_' + str(dates[i+1]) + '.csv'
        print(csv_name)

## Functions

In [5]:
def load_folder_as_dict(dates, prefix='data/'):
    dict_df = dict()
    for i in range(len(dates) - 1):
        csv_name = prefix + 'data_'+ str(dates[i]) + '_' + str(dates[i+1]) + '.csv'
        df_name = 'df_' + str(dates[i])
        dict_df[df_name] = pd.read_csv(csv_name)
    return dict_df

In [6]:
def get_stats_on_df(dict_df):
    l_1 = []
    l_2 = []

    for key in dict_df.keys():
        df = dict_df[key]
        l_user_rt = df[df.is_retweeted == True].user_retweeted.unique()
        l_users = df[df.is_retweeted == True].username.unique()
        print('Nb unique user retweeted for ', key, ' : ', len(l_user_rt))
        print('Nb unique users for ', key, ' : ', len(l_users))
        print()
        l_1.extend(l_user_rt)
        l_2.extend(l_users)

    print()
    print('Nb of unique users retweeted for al these df :', len(list(set(l_1))))
    print('Nb of unique users who tweet for al these df :', len(list(set(l_2))))
    return

In [7]:
def define_graph_from_df(df):
    G = nx.from_pandas_edgelist(
        df,
        source = 'username', 
        target = 'user_retweeted',
        create_using = nx.DiGraph())
    return G

In [8]:
def define_graphs(dict_df, stats=False):
    graph_dict = dict()
    for key in dict_df.keys():
        graph_dict[key] = define_graph_from_df(dict_df[key])
        if stats:
            print('Nodes in RT network:', len(G.nodes()))
            print('Edges in RT network:', len(G.edges()), '\n')
    return graph_dict

#### Define dates for which we have data

In [9]:
dates = ['2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31',
        '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06']

#### Define dictionaries of dataframes and graphs

In [10]:
df_dict = load_folder_as_dict(dates, prefix='data/')
len(df_dict)

9

In [11]:
graph_dict = define_graphs(df_dict)

#### Call a dataframe and graph according to a key

In [12]:
key = list(df_dict.keys())[0]
print(key)
df = df_dict[key]
df.head()

df_2020-03-28


,timestamp,username,count_rt,text,is_retweeted,user_retweeted
0,2020-03-28 23:59:40,Galvzerrr,72,RT @NoamCartozo: J'ai encore rêvé d'elle - Il ...,True,NoamCartozo
1,2020-03-28 23:58:12,SArrow9578,3907,RT @PoliceSynergie: Guet-apens cette nuit cont...,True,PoliceSynergie
2,2020-03-28 23:58:10,PhilippineGlln,0,"Donc soit ça se fait la boule à z, soit ça se ...",False,False
3,2020-03-28 23:57:37,Rausteen_,72,RT @NoamCartozo: J'ai encore rêvé d'elle - Il ...,True,NoamCartozo
4,2020-03-28 23:57:27,cyrilboji,114,RT @ScpnCommissaire: Enfin ! les crèches et le...,True,ScpnCommissaire


In [13]:
# Create retweet network from edgelist
G = nx.from_pandas_edgelist(
    df,
    source = 'username', 
    target = 'user_retweeted',
    create_using = nx.DiGraph())
    
# Print the number of nodes
print('Nodes in RT network:', len(G.nodes()))

# Print the number of edges
print('Edges in RT network:', len(G.edges()))

Nodes in RT network: 8877
Edges in RT network: 9283


In [14]:
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community.kclique import k_clique_communities
import matplotlib.pyplot as plt

In [15]:
from networkx import edge_betweenness_centrality
from random import random

def most_valuable_edge(G):
    centrality = edge_betweenness_centrality(G)
    max_cent = max(centrality.values())
    # Scale the centrality values so they are between 0 and 1,
    # and add some random noise.
    centrality = {e: c / max_cent for e, c in centrality.items()}
    # Add some random noise.
    centrality = {e: c + random() for e, c in centrality.items()}
    return max(centrality, key=centrality.get)

In [21]:
gn_generator = girvan_newman(G, most_valuable_edge)

In [22]:
from itertools import islice
gn_communities = next(islice(gn_generator, 1, None)) # Do 3 iterations only

KeyboardInterrupt: 

In [24]:
type(gn_communities)

NameError: name 'gn_communities' is not defined

In [25]:
gn_dict_communities = {}

for i, c in enumerate(gn_communities):
    print ("Community {}".format(i))
    for node in c:
        gn_dict_communities[node] = i + 1
        
for node in G:
    if node not in gn_dict_communities.keys():
        gn_dict_communities[node] = -1

NameError: name 'gn_communities' is not defined

In [23]:
gn_pos = community_layout(G, gn_dict_communities)

NameError: name 'community_layout' is not defined

In [ ]:
from matplotlib import cm
gn_colors = []
for node in G.nodes:
    gn_colors.append(cm.Set1(gn_dict_communities[node]))

In [ ]:
plt.figure(figsize=(20,20))
nx.draw_networkx_nodes(tG, gn_pos, node_color=gn_colors, node_size=20)
nx.draw_networkx_edges(tG, gn_pos, alpha=0.05)
plt.axis('off')
plt.show()